# Looping Through different variables to find the best combination

## Results
### Starting Data - 1 month
- Best Results
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for XGBoost
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for Logistic Regression
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's = 
- Best Combination for LSTM
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =

## Data
1. Data Order - Either the data is in the original order or the data is randomized after being turned into packets. This is because I believe randomizing the data will prevent overfitting better than it being in a predicable order, because I do not believe the packets should be linked, for best results they should be seen as independent by the data.
2. Underlying Data Type - Either the data is Raw or a percent change. I am not sure whether using the raw data or a decimal change of the number is better for the derivatives.
3. Number of fields - This is the number of look back periods in each packet
4. Derivatives - Using the first derivative for ROC and the second derivative for acceleration, on both the raw data as well as different TA's
5. TA's - Trend analysis that I want to use, for now we are using different EMA's and SMA's plus possibly RSI14 and Bolinger Bands, to determine if any of them will enhance the outcome

## Process
### Get the Data
1. Start with 1 month of data for speed
2. Continue with 1 year of data
3. Based on the random data vs ordered data, take 1 year of random data
    (Look at the the whole database and pull out a number of random entries equivalent to 1 years worth)
4. Take the most efficient process and apply it to the whole dataset in subgroups, randomly assigning groups. Steps 3 and 4 are to prevent overfitting over a single Ticker

### Processing 
1. I have 7 variables to test
    1. Number of Fields - Number of fields in the packet (lookback period)
    2. Derivative - Using the First and Second Derivative, and using it on:
        - Raw Data
        - Percent Change
        - EMA8, SMA8, EMA20, SMA20
        - Mix of these
    3. EMAs
    4. SMAs
    5. Other
    6. Underlying Data
    7. Data Order
2. I applied the variables that I could to the entire data set in order to modularize the code. 
3. I created a list of every combination (192 with a reduced subset of variables)
2. I looped through each combination and created the correct df with the relevant data, then I made packets and added those packets to a df for testing
3. Apply these steps to every machine learning model


# Getting Data

In [ ]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

qry = '''
    SELECT * FROM Stocks
    WHERE EXTRACT(YEAR FROM Datetime) = 2023 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

qry_month = '''
    SELECT * FROM Stocks
    WHERE Datetime >= '2023-03-01' 
    AND Datetime < '2023-04-01' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

qry_all = '''
    SELECT * FROM Stocks
    WHERE Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry_all).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)


print(df.info())

#### Selling Function

In [9]:
def trail_stoploss(row, tradingdata, whole_percentage):
    idx = row.name
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] 
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return [i, profit]

#### Trading Data - Unused bc didn't work

In [ ]:
import time

time_overall = time.time()

# Getting all of the trading days because trades can only occur during trading day
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time'] > start_time) & (df['Time'] < end_time) & (df.index >= 201)]
TradingDay_df['Profit'] = TradingDay_df.apply(trail_stoploss, axis=1, tradingdata=TradingDay_df,whole_percentage=5)

t2 = time.time()

print(f"Finished calculating Profit in {t2 - time_overall} seconds")

#applying EMAs SMAs
EMAs = [8,10,20,50,75,100]
SMAs = [8,10,20,50,75,100]
Other_tas = ["RSI14","Bolinger Bands"]


start_index = max(0,df.index.get_loc(TradingDay_df.index[0]) - 201)
end_index = start_index + len(TradingDay_df) + 202  
sub_df = df.iloc[start_index:end_index]

for EMA in EMAs:
    sub_df[f'EMA{EMA}'] = ta.ema(sub_df['close'], length=EMA)
    df.loc[sub_df.index, f'EMA{EMA}'] = sub_df[f'EMA{EMA}']

t3 = time.time()

print(f"Finished calculating EMAs in {t3 - t2} seconds")

for SMA in SMAs:
    sub_df[f'SMA{SMA}'] = ta.sma(sub_df['close'], length=SMA)
    df.loc[sub_df.index, f'SMA{SMA}'] = sub_df[f'SMA{SMA}']

t4 = time.time()

print(f"Finished calculating SMAs in {t4 - t2} seconds")

# Calculating 1st 2nd Deritivatives
# 1st
sub_df['Time_Diff'] = sub_df['Datetime'].diff().dt.total_seconds()
sub_df['dydx'] = sub_df['close'].diff() / sub_df['Time_Diff']
df.loc[sub_df.index, 'dydx'] = sub_df['dydx']

t5 = time.time()

print(f"Finished calculating 1st Derivative in {t5 - t4} seconds")

# 2nd
sub_df['dydx2'] = sub_df['dydx'].diff() / sub_df['Time_Diff']
df.loc[sub_df.index, 'dydx2'] = sub_df['dydx2']


time_end = time.time()

print(f"Finished calculating 2nd Derivative in {time_end - t5} seconds")
print(f"time taken for complete data processing = {time_end - time_overall}")

time_save = time.time()

df.to_csv('C:/Users/rybot/OneDrive/Databases/Processed_Data090324.csv', index=True)
TradingDay_df.to_csv('C:/Users/rybot/OneDrive/Databases/Processed_TradingData090324.csv', index=True)

time_savend = time.time()
print(f"Finished saving the data in f{time_savend-time_save} seconds")


## Load for Training Model
- Messed up and had to recalculate the TAs and Derivaitives so ignore above

In [23]:
def trail_stoploss(row, tradingdata, whole_percentage):
    idx = row.name
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] 
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return profit

In [24]:
import duckdb
import pandas as pd
import pandas_ta as ta
import os
import csv
import itertools

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")

# Filter data for January to March 2022
start_date = '2022-01-01'
end_date = '2022-03-31'

qry_month = f'''
    SELECT * FROM Stocks
    WHERE Datetime >= '{start_date}' 
    AND Datetime < '{end_date}' 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry_month).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)

# List of Testing Variables
Data_order = ["Random","Sorted"] # *combination[3]
#Underlying_data = ["Percent Change","Raw Data"]
Field_size = [10,20,50,100,200,300] # *combination[2]
Derivatives = [[],["dydx"],["dydx","dydx2"]] # *combination[0]
TAs = [[],["EMA"],["EMA","SMA"]] # *combination[1]

all_combinations = itertools.product(Derivatives, TAs, Field_size, Data_order)

# Getting all of the trading days because trades can only occur during trading day
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time'] > start_time) & (df['Time'] < end_time) & (df.index >= 201)]

df['Profit'] = TradingDay_df.apply(trail_stoploss, axis=1, tradingdata=df,whole_percentage=5)

#applying EMAs SMAs
EMAs = [8,10,20,50,75,100]
SMAs = [8,10,20,50,75,100]


for EMA in EMAs:
    df[f'EMA{EMA}'] = ta.ema(df['close'], length=EMA)

for SMA in SMAs:
    df[f'SMA{SMA}'] = ta.sma(df['close'], length=SMA)

# 1st
df['Time_Diff'] = df['Datetime'].diff().dt.total_seconds()
df['dydx'] = df['close'].diff() / df['Time_Diff']

# 2nd
df['dydx2'] = df['dydx'].diff() / df['Time_Diff']



In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50839 entries, 0 to 50838
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Datetime   50839 non-null  datetime64[us]
 1   open       50839 non-null  float32       
 2   high       50839 non-null  float32       
 3   low        50839 non-null  float32       
 4   close      50839 non-null  float32       
 5   volume     50839 non-null  float32       
 6   Stock      50839 non-null  object        
 7   interval   50839 non-null  float32       
 8   Time       50839 non-null  object        
 9   Profit     25552 non-null  float64       
 10  EMA8       50832 non-null  float64       
 11  EMA10      50830 non-null  float64       
 12  EMA20      50820 non-null  float64       
 13  EMA50      50790 non-null  float64       
 14  EMA75      50765 non-null  float64       
 15  EMA100     50740 non-null  float64       
 16  SMA8       50832 non-null  float64      

#### Example from the Combination Package
Random 10 None ['None']
Random 10 None ['EMA']
Random 10 None ['EMA', 'SMA']
Random 10 None ['EMA', 'Other']
Random 10 First ['None']
Random 10 First ['EMA']
Random 10 First ['EMA', 'SMA']
Random 10 First ['EMA', 'Other']
Random 10 ['First', 'Second'] ['None']
Random 10 ['First', 'Second'] ['EMA']
Random 10 ['First', 'Second'] ['EMA', 'SMA']
Random 10 ['First', 'Second'] ['EMA', 'Other']
Random 20 None ['None']
Random 20 None ['EMA']
Random 20 None ['EMA', 'SMA']
Random 20 None ['EMA', 'Other']
Random 20 First ['None']
Random 20 First ['EMA']
Random 20 First ['EMA', 'SMA']
Random 20 First ['EMA', 'Other

#### Functions

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt


def packeting(df, Trading_df, packet_size):
    rows = []
    no_profit = df.drop(columns=['Profit','Datetime','Stock','interval','open','close','high','low','volume','Time','Time_Diff'])

    columns = [f'{col}_{i}' for i in range(packet_size + 1) for col in no_profit.columns]
    columns.append('Profit')  # Add a single 'profit' column at the end

    for index, row in Trading_df.iterrows():
        if index > packet_size:
            # Create the packet, excluding 'profit'
            packet = no_profit.loc[index - packet_size:index]
            flatpacket = list(packet.values.flatten())

            current_profit = df.loc[index, 'Profit']
            flatpacket.append(current_profit) 
   
            rows.append(flatpacket)
    
    return pd.DataFrame(rows, columns=columns)




In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import itertools

XGB_params = { # From the Grid Search Previously Done
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'colsample_bytree': 0.6,
    'gamma': 0,
    'learning_rate': 0.2,
    'max_depth': 10,
    'min_child_weight': 3,
    'n_estimators': 300,
    'subsample': 0.8
}

models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor()
}

starter_df = df[['Profit']]
rmse_results = {}
   
''' 
ISSUES 
- starter_df, what to do when it is none for all of the variables you are testing = make it the original test
- Saving the RSME data
- Testing Derivatives on different EMAs/SMAs
'''

all_combinations = itertools.product(Derivatives, TAs, Field_size, Data_order)

for i, combination in enumerate(all_combinations):
    print(*combination)
    cmbo_df = starter_df.copy()
    # Derivatives
    for d in combination[0]:
        if "dxdy":
            cmbo_df['dxdy'] = df['dxdy']
        if "dxdy2":
            cmbo_df['dxdy2'] = df['dxdy2']
    # TAs
    for t in combination[1]:
        if "EMA":
            cmbo_df['EMA'] = df['EMA']
        if "SMA":
            cmbo_df['SMA'] = df['SMA']
    # Field Size
    size = combination[2]

    # Apply the packet making, possibly in a function
    
    test_df = packeting(cmbo_df,TradingDay_df,size)

    if "Random" in combination[3]:
        test_df_random = test_df.sample(frac=1).reset_index(drop=True)
        data_sets = [test_df,test_df_random]
    elif "Sorted" in combination[3]:
        data_sets = [test_df]

    for set in data_sets:
        X = set.drop(columns=['Profit'])
        y = set[['Profit']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

        y_train = y_train.values.ravel() 

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        rmse_result = {}

        # Loop through models, train, and evaluate each one
        for name, model in models.items():
            if name == 'LSTM':
                pass
            else:
                model.fit(X_train, y_train)  # Train other models
                predictions = model.predict(X_test)  # Predict on the test set

            rmse = np.sqrt(mean_squared_error(y_test, predictions))  # Calculate RMSE
            rmse_result[name] = rmse  # Store RMSE
            print(f"{name} RMSE: {rmse}")

        rmse_results[combination] = rmse_result



[] [] 10 Random
Index(['EMA8_0', 'EMA10_0', 'EMA20_0', 'EMA50_0', 'EMA75_0', 'EMA100_0',
       'SMA8_0', 'SMA10_0', 'SMA20_0', 'SMA50_0',
       ...
       'EMA100_10', 'SMA8_10', 'SMA10_10', 'SMA20_10', 'SMA50_10', 'SMA75_10',
       'SMA100_10', 'dydx_10', 'dydx2_10', 'Profit'],
      dtype='object', length=155)
X_train shape: (22996, 154)
y_train shape: (22996,)
Random Forest RMSE: 0.002001939048135626
X_train shape: (22996, 154)
y_train shape: (22996,)
Gradient Boosting RMSE: 0.004121784058519662
X_train shape: (22996, 154)
y_train shape: (22996,)
XGBoost RMSE: 0.003029774205886783


TypeError: unhashable type: 'list'

# Processing for LSTM 